In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!ls

In [ ]:
%cd gdrive/My Drive/Insubria/Magistrale/Tesi/project_folder/NicolaProject/autovc

In [ ]:
%%bash
pip install univoc pyloudnorm

In [ ]:
# make spect files (do it one times)
!python make_spect.py \
    --root-dir="./wav48" \
    --target-dir="./wav48_spmel" \
    #> make_spec.log 2>&1 &!

In [ ]:
# make new spect files (do it one times)
!python make_spect_new_mel.py \
    --root-dir="./wav48" \
    --target-dir="./wav48_spmel_new" \
    #> make_spec.log 2>&1 &!

In [ ]:
# make train.pkl file (do it one times)
!wget https://github.com/nicolalandro/autovc/releases/download/0.1/3000000-BL.ckpt CUDA_VISIBLE_DEVICES="0"  
!python make_metadata.py --root-dir="./wav48_spmel"  --dest="./wav48_spmel_new" \
    #> make_metadata.log 2>&1 &!

In [ ]:
#!wget https://github.com/nicolalandro/autovc/releases/download/0.1/autovc.ckpt CUDA_VISIBLE_DEVICES="0"  
!python main.py --data_dir="./wav48_spmel_new" \
    --outfile-path="./generator_wav48.pt" \
    --num_iters 50000 --batch_size 2 --dim_neck 32 --dim_emb 256 --dim_pre 512 --freq 32 --lambda_cd 1 --pretrained "autovc.ckpt" \
    #> train.log 2>&1 &!

In [ ]:
import csv
import pandas as pd
import matplotlib.pyplot as plt

losses = []                       # The list where we will store results.
linenum = 0
substr = "G/loss_id:".lower()          # Substring to search for.
with open ('esperimenti/lambda1.50/last_log.log', 'rt') as myfile:
    for line in myfile:
        linenum += 1
        if line.lower().find(substr) != -1:    # if case-insensitive match,
            losses.append(line.rstrip('\n'))
            
    stripped = (line.strip(',') for line in losses)
    lines = (line.split() for line in stripped if line)
    
df = pd.DataFrame(lines)
df.drop(df.columns[[0, 2, 4, 6, 8]], axis=1, inplace=True)
df.columns = ["Elapsed", "Iteration", "G/loss_id", "G/loss_id_psnt", "G/loss_cd"]
df.replace(',','', regex=True, inplace=True)
df["G/loss_id"] = pd.to_numeric(df["G/loss_id"])
df["G/loss_id_psnt"] = pd.to_numeric(df["G/loss_id_psnt"])
df["G/loss_cd"] = pd.to_numeric(df["G/loss_cd"])
#print(df)

plt.plot(df['Iteration'],df['G/loss_id'])
plt.show()